In [6]:
import collections
import json
import os
import re
import wikipedia

import ahocorasick
import openai
import pdfplumber
from pdfminer.high_level import extract_text

dataDir = "../data/"
dataName = "Deep Learning.pdf"

# 设置 api


In [7]:
openai.api_base = "https://api.chatanywhere.com.cn/v1"
# openai.api_base = 'https://api.chatanywhere.cn/v1'
openai.api_key = "sk-D1u13WweY1LhWLqv95Ml7e3y8f8ToSfsTkGnlgvSQLqZJptC"

In [8]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/Baichuan2-7B-Base", use_fast=False, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("baichuan-inc/Baichuan2-7B-Base", trust_remote_code=True)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# inputs = tokenizer('登鹳雀楼->王之涣\n夜雨寄北->', return_tensors='pt')
# inputs = inputs.to('cuda:0')
# pred = model.generate(**inputs, max_new_tokens=64, repetition_penalty=1.1)
# print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

# 一，读取数据


## 生成目录架构

- 生成**章节名称**与**章节序号**的对应：`name_dict`
- 以及**章节序号**与**章节页码**范围的对应：`index_dict`


In [9]:
with pdfplumber.open(dataDir + dataName) as f:
    # 目录架构生成
    c, p, n = [], [], []
    for i in range(7):
        page = f.pages[i]
        text = page.extract_text()
        text_split = text.split("\n")
        for i in text_split:
            if bool(re.match("[0-9]+\.[0-9]+", i.split(" ")[0])):
                c.append(i.split(" ")[0])
                p.append(i.split(" ")[-1])
            if bool(re.match("[0-9]+", i.split(" ")[0])):
                for j in i.split(" "):
                    if bool(re.match("[A-Za-z]+", j)):
                        n.append((i.split(" ")[0], j))

### name_dict


In [10]:
name_dict = {}
for i, j in n:
    if i in name_dict:
        name_dict[i] = name_dict.get(i, "") + " " + j
    else:
        name_dict[i] = name_dict.get(i, "") + j

### index_dict


In [11]:
p_range = list(zip(p, p[1:]))
p_range.append((720, 800))
c_p_range = list(zip(c, p_range))
index_dict = collections.defaultdict(list)
for k, v in c_p_range:
    index_dict[k.split(".")[0]].append((k, v))

## 生成内容表

- 段落内容表：`content_dict`


In [12]:
with pdfplumber.open(dataDir + dataName) as f:
    content_dict = collections.defaultdict(list)

    for k, v in index_dict.items():
        for i in v:
            page_range = i[-1]

            for j in range(int(page_range[0]), int(page_range[1])):
                page = f.pages[j]

                text = page.extract_text().replace('\n', ' ')

                content_dict[i[0]].append(text)

# 二，Build Relations

## 目录 + 前置

In [13]:
def catalogue_relations(dataName, name_dict=None, relation_type=["目录", "前置"]):
    c_relations = []
    p_relations = []
    if not name_dict:
        name_dict = {}
    for k, v in name_dict.items():
        if bool(re.match("[0-9]+\.[0-9]+", k)):
            p_relations.append(
                [
                    k.split(".")[0] + " " + name_dict[k.split(".")[0]],
                    relation_type[1],
                    k + " " + v,
                ]
            )
        else:
            c_relations.append(
                [
                    dataName,
                    relation_type[0],
                    k + " " + v,
                ]
            )
    return p_relations, c_relations

In [14]:
p_relations, c_relations = catalogue_relations(
    dataName=dataName.split(".")[0], name_dict=name_dict
)

## 包含 + 段落共现

### AC自动机

In [15]:
def build(patterns):
    trie = ahocorasick.Automaton()
    for index, word in enumerate(patterns):
        trie.add_word(word, (index, word))
    trie.make_automaton()
    return trie

In [57]:
data = []
for i in range(1, 21):
    with open(dataDir + 'relations/' + 'kb_chapter_' + str(i) + '.json', 'r') as f:
        data.append([i, json.load(f)])

In [58]:
for i in data:
    print(i[0], len(i[1]))

1 28
2 20
3 20
4 22
5 99
6 23
7 37
8 51
9 33
10 20
11 27
12 67
13 6
14 9
15 30
16 5
17 10
18 19
19 22
20 62


In [59]:
entity_set = set()
for i in data:
    for relation in i[1]:
        if relation[0] not in entity_set:
            entity_set.add(relation[0])
        if relation[2] not in entity_set:
            entity_set.add(relation[2])

In [62]:
num = 0
for i in entity_set:
    if not bool(re.match("^(\d+|[A-Za-z])$", i)):
        num += 1

In [63]:
num

289

In [48]:
def include_co_presence(entity_set):
    patterns = list(entity_set)
    trie = build(patterns)
    include_relations = []
    co_presence_relations = set()
    for chapter, sections in index_dict.items():
        for section in sections:
            for content in content_dict[section[0]]:
                word_set = set(word[1][1] for word in trie.iter(content))
                for head in word_set:
                    if not bool(re.match("^(\d+|[A-Za-z])$", head)):
                        include_relations.append(
                            [section[0] + " " + name_dict[section[0]], "include", head]
                        )
                        for tail in word_set:
                            if head != tail and not bool(
                                re.match("^(\d+|[A-Za-z])$", tail)
                            ):
                                if (
                                    tail,
                                    "co_presence",
                                    head,
                                ) not in co_presence_relations:
                                    co_presence_relations.add(
                                        (head, "co_presence", tail)
                                    )
    return include_relations, co_presence_relations

In [49]:
include_relations, co_presence_relations = include_co_presence(entity_set)

## 结果展示

In [50]:
p_relations[0]

['1 Introduction', '前置', '1.1 Who Should Read This Book?']

In [51]:
c_relations[0]

['Deep Learning', '目录', '1 Introduction']

In [52]:
len(co_presence_relations)

1508

In [53]:
include_relations[0]

['1.1 Who Should Read This Book?', 'include', 'Computation']

In [54]:
structure_relations = (
    p_relations + c_relations + list(co_presence_relations) + include_relations
)

In [55]:
len(structure_relations)

3080

In [56]:
with open(os.path.join(dataDir + "/relations", f"structure_relations.json"), "w") as f:
    json.dump(structure_relations, f, indent=4)

# 测试

In [ ]:
# relations = ["目录", "前置", "句子共现", "段落共现", "频繁项集"]  # pending

# conversation_list = [
#     {
#         "role": "system",
#         "content": "请只提取文本中的命名实体，格式为[{entity_A}, {entity_B}, ...], 不要返回任何其他内容",
#     }
# ]

# bot = Chat(conversation_list)
# answer = bot.ask(content_dict["1.1"][1])

In [ ]:
# res_tiny = [i.strip() for i in re.sub('"|}|{||\[|\]', "", answer).split(",")]

# 一些其他尝试

N - Gram


In [ ]:
# 暂时没有探索结果
# all_text = re.sub('[^A-Za-z0-9\.]+', ' ', text).lower().split(' ')
# ng1 = collections.defaultdict(int)
# ng2 = collections.defaultdict(int)
# ng3 = collections.defaultdict(int)
# ng4 = collections.defaultdict(int)
# for i, j in enumerate(all_text):
#     ng1[j] += 1
#     if i > 0: ng2[(all_text[i-1], j)] += 1
#     if i > 1: ng3[(all_text[i-2], all_text[i-1], j)] += 1
#     if i > 2: ng4[(all_text[i-3], all_text[i-2], all_text[i-1], j)] += 1

# def sort_feq(dic):
#     return sorted([(k, v) for k, v in dic.items()], key=lambda x: x[1], reverse=True)

### 定义 Chat 类

In [ ]:
# def total_counts(response):
#     tokens_nums = int(response["usage"]["total_tokens"])
#     price = 0.002 / 1000
#     cost = "{:.5f}".format(price * tokens_nums * 7.5)
#     print(f"tokens: {tokens_nums}, cost: {cost}")

#     return float(cost)

In [ ]:
# class Chat:
#     def __init__(self, conversation_list=[]):
#         self.conversation_list = conversation_list
#         self.costs_list = []

#     def show_conversation(self, msg_list):
#         for msg in msg_list[-2:]:
#             if msg["role"] == "user":
#                 pass
#             else:
#                 message = msg["content"]
#                 print(f"\U0001f47D: {message}\n")
#             print()

#     def ask(self, prompt):
#         self.conversation_list.append({"role": "user", "content": prompt})
#         openai.api_key = "sk-D1u13WweY1LhWLqv95Ml7e3y8f8ToSfsTkGnlgvSQLqZJptC"
#         response = openai.ChatCompletion.create(
#             model="gpt-3.5-turbo", messages=self.conversation_list
#         )
#         answer = response.choices[0].message["content"]

#         self.conversation_list.append({"role": "assistant", "content": answer})
#         self.show_conversation(self.conversation_list)

#         cost = total_counts(response)
#         self.costs_list.append(cost)
#         return answer
#         print()